In [31]:
import pandas as pd
import numpy as np


## Features engineering de pageviews

### Carga de datos

In [32]:
pageviews = pd.read_pickle("../data/pageviews_total_formateado.pkl")

---

### Funciones auxiliares

#### Creacion de ventanas iniciales

In [33]:
def generar_dataframes_ventana(numero_ventana,dataframe):
    
    if numero_ventana == 4 :   #Para la ventana 4 se necesitan los 11 mil usuarios existentes
        df = pd.read_csv("../data/users.csv", dtype = {"USER_ID": "category"}, index_col = 0)
    
    else:    
        condicion = dataframe["trimestre"] == numero_ventana
        df = dataframe[condicion]["USER_ID"].drop_duplicates().to_frame().set_index("USER_ID").copy()
    
    return df.sort_index()
    

#### Generacion de features

In [34]:
def feature_ya_generado(nombre_feature, trimestre):
    """Corrobora si un feature ya fue generado """
    
    #Obtengo las columnas que genera ese feature
    columnas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
    for columna in columnas:
        if columna in FEATURES_YA_GENERADOS[trimestre]:
            return True
    return False

In [35]:
def generar_feature_en_ventana(dataframe,trimestre, nombre_feature, destino):
    """Genera un feature en la ventana correspondiente, con los tipos de datos correspondientes"""
    
    #Calculo el feature
    funcion_feature = FUNCIONES_FEATURES[nombre_feature]
    dataframe = funcion_feature( dataframe[dataframe["trimestre"] == trimestre] )
    
    #Uno los features nuevos con los viejos
    dataframe = destino[trimestre].merge(dataframe, left_index = True, right_index= True, how = 'left')
    
    #Rello los nans, si es que los hubiera
    dataframe = dar_formato_al_feature_nuevo(dataframe,nombre_feature)
    
    destino[trimestre] = dataframe

In [36]:
def dar_formato_al_feature_nuevo(dataframe, nombre_feature):
    """Rellena los Nans de un dataframe con features nuevos"""
    
    
    #Datos para rellenar los Nan
    columnas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
    tipo_dato= DATOS_FEATURES[nombre_feature][POS_TIPO_DATO]
    valor_nan = DATOS_FEATURES[nombre_feature][POS_DATO_NAN]
    
    dataframe.fillna(valor_nan, inplace = True)
    dataframe[columnas] = dataframe[columnas].astype(tipo_dato)
    
    return dataframe
    

In [37]:
def generar_features_faltantes(dataframe,destino,trimestre):
    """
    Funcion para generar los features faltantes para un trimestre, 
    evita la repeticion de calculo de features si estos ya se encuentran generados.
    
    dataframe: Dataframe a partir de cual se calcularan los features.
    destino: Diccionario de ventanas para los que calcular los features.
    trimestre: Numero de ventana a partir de la cual se calculan los features.
    """
    
    for nombre_feature in FUNCIONES_FEATURES:    
        if feature_ya_generado(nombre_feature, trimestre):
            continue

        #Si no fueron generados, los genero y actualizo el listado:   
        generar_feature_en_ventana(dataframe,trimestre, nombre_feature, destino)   
        
        #Agrego las columnas nuevas al diccionario
        columnas_generadas = DATOS_FEATURES[nombre_feature][POS_COLUMNAS_DEL_FEATURE]
        FEATURES_YA_GENERADOS[trimestre].extend(columnas_generadas)

----

----

### Cargo features generados hasta el momento


In [38]:
# Trimestre 2 - 3 - 4 para los que tengo datos anteriores
CANT_TRIMESTRES = 4

In [39]:
TRIMESTRES = {}
trim_no_encontrados =[]

In [45]:
print("Cargando features")
for ventana_nro in range(1,CANT_TRIMESTRES) : #Trimestres del 1 al 3
    try:
        TRIMESTRES[ventana_nro] = pd.read_pickle("../data/features/ventana_{}_pageviews.pkl".format(ventana_nro))
        
    except FileNotFoundError:
        print("No se encontraron los features, se generarán los dataframe requeridos")
        trim_no_encontrados.append(ventana_nro)
    continue
    
for ventana_nro in trim_no_encontrados:    
    TRIMESTRES[ventana_nro] = generar_dataframes_ventana(ventana_nro,pageviews)

Cargando features


### Set para predecir

In [46]:
try:
    print("Cargando features para predecir")
    TRIMESTRES[4] = pd.read_pickle("../data/features/predecir_pageviews.pkl")
except FileNotFoundError:
    print("No se encontro el feature deseado, se generara el dataframe requerido")
    TRIMESTRES[4] = generar_dataframes_ventana(4,pageviews)

Cargando features para predecir


---

### Diccionario de features

In [12]:
#Posicion dentro del diccionario de features que ocupa la lista de columnas generadas por el mismo
POS_COLUMNAS_DEL_FEATURE = 0
POS_TIPO_DATO = 1
POS_DATO_NAN = 2

In [13]:
# Clave = nombre_feature : Valor = funcion
FUNCIONES_FEATURES = {}
# Clave = nombre_feature : Valor = (columnas,tipo_dato,valor_nan)
DATOS_FEATURES = {}

In [14]:
FEATURES_YA_GENERADOS = {}

for ventana_nro in range (1, CANT_TRIMESTRES + 1):
    
    FEATURES_YA_GENERADOS[ventana_nro] = list(TRIMESTRES[ventana_nro].columns.values)

---


---

## Creación de features

---

#### Cantidad de paginas vista por un usuario en todo el trimestre

In [15]:
nombre = "cantidad paginas vistas"

In [16]:
def cantidad_de_paginas_vistas(dataframe):
    
    feature = dataframe.groupby("USER_ID").agg({"FEC_EVENT":"count"})\
                        .rename(columns = {"FEC_EVENT":"cantidad_paginas_vistas_trimestre"}) 
    return feature

In [17]:
FUNCIONES_FEATURES[nombre] = cantidad_de_paginas_vistas

In [18]:
columnas_generadas = ["cantidad_paginas_vistas_trimestre"] #lista
tipo_dato = "int64"
dato_nan = 0

DATOS_FEATURES[nombre] = (columnas_generadas, tipo_dato, dato_nan)

---

#### Cantidad de paginas vista dentro de cada mes dentro del trimestre

In [19]:
nombre = "cantidad paginas vistas mes"

In [20]:
def cantidad_paginas_vistas_por_mes(dataframe):
    
    vistas_por_mes = dataframe.groupby(["USER_ID","mes"]).agg({"trimestre":"count"})
    vistas_por_mes.reset_index(inplace = True)
    vistas_por_mes = vistas_por_mes.pivot(index = "USER_ID", columns = "mes", values = "trimestre")
    
    vistas_por_mes.columns = ["cant_paginas_vistas_en_mes_{}".format(mes) for mes in range(1,4)]

    return vistas_por_mes

In [21]:
FUNCIONES_FEATURES[nombre] = cantidad_paginas_vistas_por_mes

In [22]:
columnas_generadas = ["cant_paginas_vistas_en_mes_{}".format(mes) for mes in range(1,4)]
tipo_dato = "int64"
dato_nan = 0

DATOS_FEATURES[nombre] = (columnas_generadas,tipo_dato,dato_nan) 

---

#### Page mas vista cada mes

In [23]:
nombre = "page mas vista mes"

In [24]:
def page_mas_vista_cada_mes(dataframe):
    
    feature = dataframe.groupby(['USER_ID','mes','PAGE']).agg({'trimestre':'count'})
    index = feature.groupby(['USER_ID','mes'])['trimestre'].transform(max) == feature['trimestre']
    feature = feature[index]
    
    #Elimino duplicados
    feature.reset_index(inplace=True)
    feature = feature.drop_duplicates(subset=['USER_ID','mes'])  
    
    #Cambio la forma del dataframe -> indice = USER_ID, columnas = "meses (1,2,3), valores = "pagina"
    feature = feature.pivot(index = "USER_ID",columns = "mes", values = "PAGE")
    
    #lleno los meses vacios con -1
    feature.columns = [x for x in range(1,4)]
    for mes in range(1,4):
        try:             #si no existe el mes, lo creo
            feature[mes] = feature[mes].cat.add_categories([-1]).fillna(-1)   #agrego la categoria -1
        except KeyError:
            feature.insert(mes,mes,-1)

                
    feature.columns = ["page_mas_vista_en_mes_{}".format(mes) for mes in range(1,4)]
    
    return feature

In [25]:
FUNCIONES_FEATURES[nombre] = page_mas_vista_cada_mes

In [26]:
columnas_generadas = ["page_mas_vista_en_mes_{}".format(mes) for mes in range(1,4)]
tipo_dato = "category"
dato_nan = -1

DATOS_FEATURES[nombre]=(columnas_generadas, tipo_dato, dato_nan)

---

## Generacion de features

In [27]:
for ventana_nro in range(1, CANT_TRIMESTRES + 1):
    generar_features_faltantes(pageviews, TRIMESTRES, ventana_nro)

In [42]:
TRIMESTRES[1].head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3,page_mas_vista_en_mes_1,page_mas_vista_en_mes_2,page_mas_vista_en_mes_3
USER_ID,,,,,,,
0,1841,710,375,756,10,10,10
1,623,250,192,181,10,2,10
10,174,74,77,23,10,10,1
100,904,375,206,323,2,10,10
1000,302,0,236,66,-1,10,3


In [43]:
TRIMESTRES[4].head()

,cantidad_paginas_vistas_trimestre,cant_paginas_vistas_en_mes_1,cant_paginas_vistas_en_mes_2,cant_paginas_vistas_en_mes_3,page_mas_vista_en_mes_1,page_mas_vista_en_mes_2,page_mas_vista_en_mes_3
USER_ID,,,,,,,
0,1519,527,490,502,2,2,2
1,477,138,140,199,2,2,2
10,21,12,2,7,2,2,2
100,1969,498,731,740,3,2,2
1000,645,215,243,187,2,2,3


### Guardado

In [44]:
for ventana_nro in range(1, CANT_TRIMESTRES ):
    TRIMESTRES[ventana_nro].to_pickle("../data/features/ventana_{}_pageviews.pkl".format(ventana_nro))

TRIMESTRES[4].to_pickle("../data/features/predecir_pageviews.pkl")

---